In [1]:
import pandas as pd
import duckdb 
import sqlite3

In [2]:
database_name ="../test_analytics.db"


In [3]:
def returning_df_from_all_tables(query):
	try:

		conn = sqlite3.connect(database_name)
		cursor = conn.cursor()

		cursor.execute (query)
		tables = [table[0] for table in cursor.fetchall()]

		if not tables:
			print ("Not is possible")

		union_all_query = " UNION ALL ".join(f"SELECT * FROM \"{table}\"" for table in tables)
		#cursor.execute(union_all_query)

		# rows = cursor.fetchall()
		df = pd.read_sql_query(union_all_query, conn)

		return df

	except sqlite3.Error as e:
		print(f"SQLITE error: {e}")

	finally:
		if conn:
			conn.close()


query =   """
  SELECT name FROM sqlite_master
  WHERE type = 'table' AND  (name LIKE 'escolas%' OR name LIKE '%escolas%');
  """

query2 = """
  SELECT name FROM sqlite_master
  WHERE type = 'table' AND  (name LIKE 'idadeserie%' OR name LIKE '%idadeserie%');
  """

df_escolas = returning_df_from_all_tables(query)

df_estudantes = returning_df_from_all_tables(query2)

In [4]:
df_estudantes = returning_df_from_all_tables(query2)

# Data Cleaning Escolas

In [5]:
df_escolas.head()

,dre,codesc,tipoesc,nomesc,nomescof,ceu,diretoria,subpref,endereco,numero,...,dom_criacao,dt_ini_func,dt_ini_conv,dt_autoriza,dt_extincao,nome_ant,rede,latitude,longitude,database
0,BT,307207,ESC.PART.,A HEBRAICA,A HEBRAICA,None,BUTANTA,PINHEIROS,HUNGRIA,1000,...,01/02/2010,None,None,06/02/2010,None,None,PAR,-23578930,-46694078,31/12/2010
1,BT,307412,ESC.PART.,ABCD SOFIA,ABCD SOFIA,None,BUTANTA,BUTANTA,OTÁVIO PEDREIRO ROSA,212,...,10/11/2010,None,None,12/11/2010,None,None,PAR,-23583388,-46760545,31/12/2010
2,BT,307131,ESC.PART.,"ACONCHEGO, ESCOLA","ACONCHEGO, ESCOLA",None,BUTANTA,BUTANTA,QUITANDUBA,50,...,21/08/2009,None,None,None,None,None,PAR,-23583061,-46715523,31/12/2010
3,BT,306549,ESC.PART.,ADOLPHE FERRIERE,ADOLPHE FERRIERE,None,BUTANTA,BUTANTA,RUA FRANCISCO PRETO,457,...,None,None,None,21/10/2003,None,None,PAR,-23603215,-46726799,31/12/2010
4,BT,92754,EMEF,"ALCIDES GONCALVES ETCHEGOYEN, GEN.","ALCIDES GONCALVES ETCHEGOYEN, GEN.",None,BUTANTA,BUTANTA,RUA ADHERBAL STRESSER,686,...,16/02/1971,None,None,10/04/1981,None,ARPOADOR,DIR,-23594177,-46791273,31/12/2010


In [6]:
df_escolas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63857 entries, 0 to 63856
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dre          62607 non-null  object 
 1   codesc       63857 non-null  int64  
 2   tipoesc      63857 non-null  object 
 3   nomesc       63857 non-null  object 
 4   nomescof     37254 non-null  object 
 5   ceu          2117 non-null   object 
 6   diretoria    62607 non-null  object 
 7   subpref      62650 non-null  object 
 8   endereco     63857 non-null  object 
 9   numero       63857 non-null  object 
 10  bairro       63857 non-null  object 
 11  cep          63857 non-null  int64  
 12  tel1         52075 non-null  object 
 13  tel2         14836 non-null  object 
 14  fax          10273 non-null  object 
 15  situacao     63857 non-null  object 
 16  coddist      62136 non-null  float64
 17  distrito     62649 non-null  object 
 18  setor        61982 non-null  float64
 19  codi

In [7]:
df_escolas.isnull().sum()

dre             1250
codesc             0
tipoesc            0
nomesc             0
nomescof       26603
ceu            61740
diretoria       1250
subpref         1207
endereco           0
numero             0
bairro             0
cep                0
tel1           11782
tel2           49021
fax            53584
situacao           0
coddist         1721
distrito        1208
setor           1875
codinep         6447
codcie          7082
eh             41765
fx_etaria      40963
dt_criacao     15290
ato_criacao    15301
dom_criacao    15320
dt_ini_func    55337
dt_ini_conv    45847
dt_autoriza    22055
dt_extincao    63770
nome_ant       35264
rede           16187
latitude        1704
longitude       1704
database        1207
dtype: int64

In [8]:
df_escolas.nunique()

dre               27
codesc          8671
tipoesc           64
nomesc          9671
nomescof        7262
ceu               59
diretoria         26
subpref           35
endereco       14481
numero          2846
bairro          2415
cep             5852
tel1           12444
tel2            3425
fax             1633
situacao           3
coddist           98
distrito          96
setor            463
codinep         6294
codcie         16121
eh              1765
fx_etaria        519
dt_criacao      8021
ato_criacao     4234
dom_criacao     7874
dt_ini_func     3122
dt_ini_conv     3296
dt_autoriza     4869
dt_extincao       42
nome_ant        6502
rede               3
latitude       18329
longitude      18366
database          14
dtype: int64

In [74]:
clean_df_escolas = df_escolas.copy()

In [75]:
clean_df_escolas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63857 entries, 0 to 63856
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dre          62607 non-null  object 
 1   codesc       63857 non-null  int64  
 2   tipoesc      63857 non-null  object 
 3   nomesc       63857 non-null  object 
 4   nomescof     37254 non-null  object 
 5   ceu          2117 non-null   object 
 6   diretoria    62607 non-null  object 
 7   subpref      62650 non-null  object 
 8   endereco     63857 non-null  object 
 9   numero       63857 non-null  object 
 10  bairro       63857 non-null  object 
 11  cep          63857 non-null  int64  
 12  tel1         52075 non-null  object 
 13  tel2         14836 non-null  object 
 14  fax          10273 non-null  object 
 15  situacao     63857 non-null  object 
 16  coddist      62136 non-null  float64
 17  distrito     62649 non-null  object 
 18  setor        61982 non-null  float64
 19  codi

In [76]:
columns_to_drop = ["ceu","tel1","tel2", "fax","codinep","codcie","eh","ato_criacao","dom_criacao", "dt_extincao","nome_ant","nomescof"]
date_columns = ["dt_criacao","dt_ini_func", "dt_ini_conv","dt_autoriza","database"]

In [77]:
clean_df_escolas = clean_df_escolas.drop(columns=columns_to_drop)

In [78]:
for col in date_columns:
  clean_df_escolas[col] = pd.to_datetime(clean_df_escolas[col], errors='coerce')

C:\Users\davi_\AppData\Local\Temp\ipykernel_19408\628194066.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  clean_df_escolas[col] = pd.to_datetime(clean_df_escolas[col], errors='coerce')
C:\Users\davi_\AppData\Local\Temp\ipykernel_19408\628194066.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  clean_df_escolas[col] = pd.to_datetime(clean_df_escolas[col], errors='coerce')


In [79]:
clean_df_escolas = clean_df_escolas.dropna(subset="database")

In [80]:
clean_df_escolas.isna().sum()

dre                0
codesc             0
tipoesc            0
nomesc             0
diretoria          0
subpref            0
endereco           0
numero             0
bairro             0
cep                0
situacao           0
coddist          456
distrito           1
setor            610
fx_etaria      28131
dt_criacao     32933
dt_ini_func    44206
dt_ini_conv    35253
dt_autoriza    36433
rede           13132
latitude         441
longitude        441
database           0
dtype: int64

In [82]:
df_estudantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2982929 entries, 0 to 2982928
Data columns (total 19 columns):
 #   Column     Dtype  
---  ------     -----  
 0   dre        object 
 1   codesc     int64  
 2   tipoesc    object 
 3   nomesc     object 
 4   distrito   object 
 5   setor      int64  
 6   ano        int64  
 7   rede       object 
 8   modal      object 
 9   descserie  object 
 10  periodo    object 
 11  turno      int64  
 12  descturno  object 
 13  sexo       object 
 14  idade      float64
 15  nee        object 
 16  raca       object 
 17  qtde       int64  
 18  database   object 
dtypes: float64(1), int64(5), object(13)
memory usage: 432.4+ MB


In [50]:
cols_endereco = ["codesc","endereco","cep","bairro","numero","coddist","setor","latitude","longitude","database"]

In [68]:
test = 306291

In [69]:
mask = clean_df_escolas["codesc"] == test
clean_df_escolas[mask].loc[:,cols_endereco]

,codesc,endereco,cep,bairro,numero,coddist,setor,latitude,longitude,database
868,306291,AVENIDA PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,-23459692,-46648590,2010-12-31
4375,306291,AVENIDA PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,-23459692,-46648590,2011-12-31
8020,306291,AVENIDA PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,-23460000,-46648435,2012-12-27
11782,306291,AVENIDA PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,-23460000.0,-46648435.0,2013-11-30
15642,306291,AVENIDA PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,-23460000.0,-46648435.0,2014-12-31
19704,306291,AVENIDA PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,-46648435.0,-23460000.0,2015-12-31
23990,306291,Avenida PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,#CAMPO!,-46.648.435,2018-12-30
28799,306291,Avenida PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,-23.46,-46.648.435,2019-12-01
43358,306291,AVENIDA PERI RONCHETTI,2633000,JARDIM PERI,25,13.0,1302.0,-23460000,-46648435,2022-12-31
48347,306291,AVENIDA PERI RONCHETTI,2633000,JARDIM PERI,25,13.0,1302.0,-23460000,-46648435,2023-12-25


In [67]:
mask = clean_df_escolas["latitude"] == "#CAMPO!"
clean_df_escolas[mask].loc[:,cols_endereco]

,codesc,endereco,cep,bairro,numero,coddist,setor,latitude,longitude,database
23990,306291,Avenida PERI RONCHETTI,2633000,JARDIM PERI,168,13.0,1302.0,#CAMPO!,-46.648.435,2018-12-30
